In [1187]:
from char_rnn import *

In [1188]:
import char_rnn
reload(char_rnn)
from char_rnn import *

In [1189]:
with open("tiny_shakespeare.txt", 'r') as f:
    text = f.read()

# text = text[:10] * 1000
print(text[100:200])
print(len(text))
text = text[:1000]
print(text)

 are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our

In [1190]:
# prepare data
train_size = int(0.8 * len(text))
valid_size = int(0.1 * len(text))
test_size = len(text) - train_size - valid_size
train_text = text[:train_size]
valid_text = text[train_size:train_size + valid_size]
test_text = text[train_size + valid_size:]

print(train_size, train_text[:64])
print(valid_size, valid_text[:64])
print(test_size, test_text[:64])

(800, 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAl')
(100, 'articularise their abundance; our\nsufferance is a gain to them L')
(100, 're we become rakes: for the gods know I\nspeak this in hunger for')


In [1191]:
# unique_chars = text[:10] # list(set(text))
unique_chars = list(set(text))
vocab_size = len(unique_chars)
print('vocab size: %d' % vocab_size)
vocab_index_dict = {}
index_vocab_dict = {}

for i, char in enumerate(unique_chars):
    vocab_index_dict[char] = i
    index_vocab_dict[i] = char

vocab size: 46


In [1192]:
vocab_index_dict

{'\n': 0,
 ' ': 2,
 '!': 1,
 "'": 3,
 ',': 4,
 '.': 5,
 ':': 7,
 ';': 6,
 '?': 8,
 'A': 9,
 'B': 11,
 'C': 10,
 'F': 12,
 'I': 13,
 'L': 15,
 'M': 14,
 'N': 17,
 'O': 16,
 'R': 19,
 'S': 18,
 'W': 20,
 'Y': 21,
 'a': 22,
 'b': 24,
 'c': 23,
 'd': 26,
 'e': 25,
 'f': 28,
 'g': 27,
 'h': 30,
 'i': 29,
 'j': 32,
 'k': 31,
 'l': 34,
 'm': 33,
 'n': 36,
 'o': 35,
 'p': 37,
 'r': 39,
 's': 38,
 't': 41,
 'u': 40,
 'v': 43,
 'w': 42,
 'y': 44,
 'z': 45}

In [1193]:
batch_size = 10
n_unrollings = 10
train_batches = BatchGenerator(train_text, batch_size, n_unrollings, vocab_size, 
                               vocab_index_dict, index_vocab_dict)
eval_train_batches = BatchGenerator(train_text, 1, 1, vocab_size,
                                    vocab_index_dict, index_vocab_dict)
valid_batches = BatchGenerator(valid_text, 1, 1, vocab_size,
                               vocab_index_dict, index_vocab_dict)
test_batches = BatchGenerator(test_text, 1, 1, vocab_size,
                              vocab_index_dict, index_vocab_dict)

In [1194]:
print(batches2string(train_batches.next(), index_vocab_dict))
print(batches2string(valid_batches.next(), index_vocab_dict))

['First Citiz', '\n\nFirst Cit', 'ed. resolve', 'e people.\n\n', ' have corn ', 'be done: aw', 'e are accou', 'ould reliev', 'some, we mi', 'the leannes']
['ar']


In [1195]:
params = {'batch_size': batch_size, 'num_unrollings': n_unrollings, 'vocab_size': vocab_size, 
        'hidden_size': 100, 'max_grad_norm': 5.0, 'embedding_size': 50, 
        'num_layers': 1}
tf.variable_scope

<function tensorflow.python.ops.variable_scope.variable_scope>

In [1196]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('char_rnn') as scope:
        train_model = CharRNN(is_training=True, **params)
        tf.get_variable_scope().reuse_variables()
        valid_model = CharRNN(is_training=False, **params)
        test_model = CharRNN(is_training=False, **params)

In [1199]:
n_epochs = 30
model_name = 'char_rnn'
saved_model_dir = '/tmp/char_rnn/char_rnn'
init_from_path = '' #saved_path #'/tmp/saved_char_rnn_model-240'
log_dir = '/tmp/char_rnn_logs'

In [1200]:
with tf.Session(graph=graph) as session:
    writer = tf.train.SummaryWriter(log_dir, session.graph_def)
    if init_from_path:
        train_model.saver.restore(session, init_from_path)
    else:
        tf.initialize_all_variables().run()
    for i in range(n_epochs):
        print('\nEpoch %d\n' % i)
        print('training')
        train_model.run_epoch(session, train_size, train_batches, is_train=True, verbose=False, summary_writer=writer)

        saved_path = train_model.saver.save(session, saved_model_dir, 
                                            global_step=train_model.global_step)
        print('model saved in %s\n' % saved_path)
        print('validation')
        valid_model.run_epoch(session, valid_size, valid_batches, 
                              is_train=False, verbose=False)
    print('\ntest')
    test_model.run_epoch(session, test_size, test_batches, 
                         is_train=False, verbose=True)



Epoch 0

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 53.476, speed: 381 wps
model saved in /tmp/char_rnn/char_rnn-7

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 38.064, speed: 1070 wps

Epoch 1

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 34.123, speed: 467 wps
model saved in /tmp/char_rnn/char_rnn-14

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 24.159, speed: 1099 wps

Epoch 2

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 26.528, speed: 461 wps
model saved in /tmp/char_rnn/char_rnn-21

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 22.252, speed: 1099 wps

Epoch 3

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 25.680, speed: 477 wps
model saved in /tmp/char_rnn/char_rnn-28

validation
epoch_size: 99
data_size: 100
num_

In [1201]:
train_model.zero_state
saved_path

'/tmp/char_rnn/char_rnn-210'

In [1204]:
with tf.Session(graph=graph) as session:
    train_model.saver.restore(session, saved_path)
    print(valid_model.sample_seq(session, 400, vocab_index_dict, index_vocab_dict, max_prob=True,
                                 prime_text=train_text[:10]))
    # "First Citizen:\nWe are accounted poor "))

First Citizen:

Firs the are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are a


In [1137]:
print(train_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [1166]:
# try_text = '10987098709'
# try_text = '12345678901'
try_text = 'First sissssssssssss'
try_text = train_text[:20]
# try_text = valid_text[80:100]
try_batches = BatchGenerator(try_text, 1, 1, vocab_size, vocab_index_dict, index_vocab_dict)

In [1167]:
with tf.Session(graph=graph) as sess:
    train_model.saver.restore(sess, saved_path)
    print(valid_model.run_epoch(sess, len(try_text), try_batches,
                        is_train=False, verbose=True, debug=False))

epoch_size: 19
data_size: 20
num_unrollings: 1
batch_size: 1
47.4%, step:9, perplexity: 1.052, speed: 400 wps
final ppl: 1.078, speed: 602 wps
1.07818488991


In [1050]:
train_text[:20]

'First CitiFirst Citi'

In [1051]:
index_vocab_dict

{0: ' ', 1: 'C', 2: 'F', 3: 'i', 4: 's', 5: 'r', 6: 't'}

In [1052]:
vocab_index_dict

{' ': 0, 'C': 1, 'F': 2, 'i': 3, 'r': 5, 's': 4, 't': 6}

In [1053]:
[index_vocab_dict[int(t)] for t in try_text]

ValueError: invalid literal for int() with base 10: 'F'

In [ ]:
a = [1]

In [ ]:
a[:-1]

In [ ]:
print('a %s' % 1)

In [ ]:
a = np.array([[1]])
a[0, 0]